-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
%load_ext line_profiler
import sys
sys.path.append("..")

In [31]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
from numpy.linalg import matrix_power
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [4]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [5]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [6]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [21]:
# %%prun

Emax = 1e20
Emin = 1e-8
nEe = 5000
nEp  = 5000

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

Emax = 1e10
Emin = 1e-8
nEe = 5000
nEp  = 5000

dlnEp = np.log(Emax/Emin)/nEp
lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

dlnEe = np.log(Emax/Emin)/nEe
lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

print('********* Thomson regime scattered photon spectrum *********')
ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400))
print('********* Relativistic regime scattered photon spectrum *********')
ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(400), inf_upp_bound=True)
print('********* Thomson regime energy loss spectrum *********')
engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400), nonrel=True)

********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [22]:
pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "wb"))
pickle.dump(ics_rel_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "wb"))
pickle.dump(engloss_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "wb"))


In [23]:
ics_thomson_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "rb"))
ics_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "rb"))
engloss_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "rb"))


In [24]:
# test = ics_rel_ref_tf_2[20]

In [25]:
%%prun
start_rs = 3000
rs = start_rs
while rs > 2500:
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    rs = np.exp(np.log(rs) - 0.002)
    print(rs)

2994.005996001995
2988.023968031968
2982.053892161797
2976.095744511174
2970.149501247497
2964.215138585785
2958.29263278858
2952.38196016585
2946.4830970748976
2940.5960199202623
2934.720705153627
2928.8571292737256
2923.0052688262463
2917.1651004037394
2911.336600645524
2905.5197462375927
2899.714513912521
2893.9208804493705
2888.1388226736008
2882.3683174569724
2876.609341717457
2870.8618724191438
2865.1258865721484
2859.4013612325193
2853.6882735021477
2847.9866005286754
2842.2963195054017
2836.6174076711964
2830.9498423104037
2825.293600752755
2819.648660373276
2814.0149985921985
2808.392592874867
2802.781420731652
2797.1814597178563
2791.5926874336296
2786.0150815238744
2780.4486196781604
2774.893279630632
2769.349039159922
2763.8158760890615
2758.29376828539
2752.7826936604692
2747.282630169994
2741.7935558137024
2736.3154486352896
2730.8482867223197
2725.3920482061376
2719.9467112617826
2714.512254107899
2709.0886550066516
2703.675892263636
2698.2739442277943
2692.8827892913273

In [12]:
%lprun -f spectools.engloss_rebin_fast spectools.engloss_rebin_fast(lowengEe_rel,lowengEp_rel,ics_rel_ref_tf_2.grid_vals,lowengEe_rel)

## Evolution

In [26]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**12*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    eleceng)

elec_spec_init.switch_spec_type()

In [27]:
print(phot_spec_init.toteng()/mDM)
print(elec_spec_init.toteng()/mDM + 2*phys.me/mDM)

0.9999999999999996
1.0000000000000002


In [32]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 4

sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz*coarsen_factor/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dlnz * coarsen_factor /phys.hubble(rs) / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    elec_processes = False
    
    if in_spec_elec.totN() > 0:
        elec_processes = True
    
    if elec_processes:
        (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
            ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
            eleceng, photeng, rs, fast=True
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()
        
#         print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
#         print('ICS secondary photon spectrum energy: ', ics_phot_spec.toteng()*norm_fac(rs))
#         print('Upscattered photon energy: ', np.dot(in_spec_elec.N, continuum_loss)*norm_fac(rs))
#         print('ICS low energy electron spectrum energy: ', (ics_lowengelec_spec.toteng() + phys.me)*norm_fac(rs))
#         print('Positronium spectrum energy: ', positronium_phot_spec.toteng()*norm_fac(rs))
    
        init_inj_spec = (in_spec_phot + ics_phot_spec + positronium_phot_spec)*norm_fac(rs)
    else:
        init_inj_spec = in_spec_phot * norm_fac(rs)
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # For the tqdm bar
    pbar = tqdm(total = np.floor((np.log(rs) - np.log(end_rs))/dlnz/coarsen_factor).astype(int))
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # Update tqdm bar.
        pbar.update(1)
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )  
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
        if coarsen_factor > 1:
            prop_tf = np.zeros_like(highengphot_tf._grid_vals)
            for i in np.arange(coarsen_factor):
                prop_tf += matrix_power(highengphot_tf._grid_vals, i)
            lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
            lowengelec_tf._grid_vals = np.matmul(prop_tf, lowengelec_tf._grid_vals)
            highengphot_tf._grid_vals = matrix_power(
                highengphot_tf._grid_vals, coarsen_factor
            )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])

        if elec_processes:
            next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec*norm_fac(rs)
        else:
            next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1])
        
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002*coarsen_factor)
        
        dt = dlnz*coarsen_factor/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        if elec_processes:
            (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
                ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
                eleceng, photeng, rs, fast=True
            )

            ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec)
            ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec)

            next_inj_spec = (in_spec_phot + ics_phot_spec + positronium_phot_spec)*norm_fac(rs)
    
        else:
            next_inj_spec = in_spec_phot * norm_fac(rs)
        

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [34]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=1, std_soln=True
    )

x_e at 3000: 0.999999999999
Standard x_e at 3000: 0.999999999999
T_m at 3000: 0.70459598745
Standard T_m at 3000: 0.70459598745
************************ END OF LOOP **************************


../darkhistory/spec/spectrum.py:758: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
../darkhistory/spec/spectrum.py:663: RuntimeWarning: divide by zero encountered in log
  np.log(bound_arr),


x_e at 2994.005996001995: 0.9999999971389908
Standard x_e at 2994.005996001995: 0.9999999971389522
T_m at 2994.005996001995: 0.7031880248258764
Standard T_m at 2994.005996001995: 0.7031880248257363
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [1.6899850251924653e-06, 1.0187568495223157e-05, 5.7508427856692274e-05, 6.740728352169611e-19, 0.0006740728334118254]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [1.6899850251924653e-06, 1.0187568495223157e-05, 5.7508427856692274e-05, 6.740779981335854e-19, 0.0006740728334118254]
************************ END OF LOOP **************************
x_e at 2988.023968031968: 0.9999999970356837
Standard x_e at 2988.023968031968: 0.9999999970348642
T_m at 2988.023968031968: 0.7017830533982116
Standard T_m at 2988.023968031968: 0.7017830533974218
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [5.1899768889104735e-05, 0.00030209314474788097, 0.0010842103977869388, 3.79821262606714e-18, 0.003798186323395415]
Standard f_cont, f_l

************************ END OF LOOP **************************
x_e at 2911.336600645524: 0.9999999952731735
Standard x_e at 2911.336600645524: 0.9999999952478977
T_m at 2911.336600645524: 0.6837718435307812
Standard T_m at 2911.336600645524: 0.6837718435155297
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.001273629223634612, 0.007064978813814989, 0.022942099674437836, 6.61377680297907e-17, 0.06613705339291358]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.001273629223634612, 0.007064978813814989, 0.022942099674437836, 6.61377984650645e-17, 0.06613705339291358]
************************ END OF LOOP **************************
x_e at 2905.5197462375927: 0.9999999950976353
Standard x_e at 2905.5197462375927: 0.9999999950695925
T_m at 2905.5197462375927: 0.6824056656287549
Standard T_m at 2905.5197462375927: 0.6824056656120984
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0013765785686868576, 0.00763010461444434, 0.02476245091573574, 7.165332544071572e-1

x_e at 2830.9498423104037: 0.9999999920573259
Standard x_e at 2830.9498423104037: 0.9999999919829702
T_m at 2830.9498423104037: 0.6648917742784195
Standard T_m at 2830.9498423104037: 0.6648917742417936
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0025430782259636318, 0.014071693346795245, 0.04589923312812931, 1.420623920434006e-16, 0.14205978544403214]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0025430782259636318, 0.014071693346795245, 0.04589923312812931, 1.4206251748674107e-16, 0.14205978544403214]
************************ END OF LOOP **************************
x_e at 2825.293600752755: 0.9999999917515188
Standard x_e at 2825.293600752755: 0.9999999916727261
T_m at 2825.293600752755: 0.6635633187476536
Standard T_m at 2825.293600752755: 0.663563318709406
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0026165670650483367, 0.014482335343027257, 0.0472806885691636, 1.4714770868221555e-16, 0.1471449028291203]
Standard f_cont, f_lyman, f_ionH, f_io

...

************************ END OF LOOP **************************
x_e at 1086.1206256305802: 0.19107394112261278
Standard x_e at 1086.1206256305802: 0.19107027087961828
T_m at 1086.1206256305802: 0.2550944728231718
Standard T_m at 1086.1206256305802: 0.2550944612517254
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005985154486012006, 0.020222540364675183, 0.05499538389244335, 0.0011133657964351787, 0.1860604063363557]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.006118691161236447, 0.020505254849635474, 0.05529694264180527, 0.0011352699761540425, 0.1853206922470902]
************************ END OF LOOP **************************
x_e at 1083.9505551731334: 0.1858658968485914
Standard x_e at 1083.9505551731334: 0.18586159556861492
T_m at 1083.9505551731334: 0.25458477041962585
Standard T_m at 1083.9505551731334: 0.2545847585178174
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.006104855919549064, 0.020454576997692604, 0.05515368858339366, 0.001132997574

x_e at 1056.1310613060716: 0.12874829235319324
Standard x_e at 1056.1310613060716: 0.1287453738818305
T_m at 1056.1310613060716: 0.24805030101480055
Standard T_m at 1056.1310613060716: 0.24805028341409624
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.00766814119758442, 0.023514218209703425, 0.05764067310443125, 0.0013915120792846341, 0.17043745957201506]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.007772707861959536, 0.02375700933668711, 0.05801564444914738, 0.0014130761590966058, 0.16969356635612817]
************************ END OF LOOP **************************
x_e at 1054.0209100381112: 0.12510537114082826
Standard x_e at 1054.0209100381112: 0.1251015208336136
T_m at 1054.0209100381112: 0.24755462802706923
Standard T_m at 1054.0209100381112: 0.24755460990607306
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.007754119473824423, 0.02369661427908037, 0.05786388937996586, 0.001409854505674702, 0.16936745062536618]
Standard f_cont, f_lyman, f_ionH, 

************************ END OF LOOP **************************
x_e at 1024.9176712863261: 0.08339907451297957
Standard x_e at 1024.9176712863261: 0.0833958636885577
T_m at 1024.9176712863261: 0.24071786285563607
Standard T_m at 1024.9176712863261: 0.24071783321516751
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.009108594978383913, 0.026706906455034665, 0.0613139818322593, 0.0016820147389851987, 0.15411853383470794]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.009237956870213986, 0.02702406248299196, 0.061693728319592966, 0.0017061064456195547, 0.15326817772095253]
************************ END OF LOOP **************************
x_e at 1022.8698844132224: 0.08100469172685915
Standard x_e at 1022.8698844132224: 0.08100175497279198
T_m at 1022.8698844132224: 0.24023677266635773
Standard T_m at 1022.8698844132224: 0.24023674151191846
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.009219558112935015, 0.026968406336389788, 0.06156458772027325, 0.00170271

x_e at 996.618020487851: 0.05544712524284129
Standard x_e at 996.618020487851: 0.055444551804311445
T_m at 996.618020487851: 0.23406881299109272
Standard T_m at 996.618020487851: 0.23406878322219968
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.010740881252825801, 0.03061589207975491, 0.06501335347860504, 0.0019800033849286403, 0.13843522907682063]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.010884899946851956, 0.03097788795958622, 0.06542867442589742, 0.002005846304566764, 0.13748805063603264]
************************ END OF LOOP **************************
x_e at 994.6267763547567: 0.0538549091755155
Standard x_e at 994.6267763547567: 0.053852275663327856
T_m at 994.6267763547567: 0.23360091354067206
Standard T_m at 994.6267763547567: 0.23360086972407834
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.010869236922246475, 0.03092704238605553, 0.0653026860751582, 0.002003407272843454, 0.13728936449182677]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_

KeyboardInterrupt: 

In [ ]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])